In [1]:
import numpy as np
import pandas as pd
train=pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test=pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
greeks=pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
train_result=train['Class']
train.pop('Class')
train=train.interpolate(method='cubic',limit_direction='forward')
print(train)

               Id        AB          AF          AH          AM         AR  \
0    000ff2bfdfe9  0.209377  3109.03329   85.200147   22.394407   8.138688   
1    007255e47698  0.145282   978.76416   85.200147   36.968889   8.138688   
2    013f2bd269f5  0.470030  2635.10654   85.200147   32.360553   8.138688   
3    043ac50845d5  0.252107  3819.65177  120.201618   77.112203   8.138688   
4    044fb8a146ec  0.380297  3733.04844   85.200147   14.103738   8.138688   
..            ...       ...         ...         ...         ...        ...   
612  fd3dafe738fd  0.149555  3130.05946  123.763599    9.513984  13.020852   
613  fd895603f071  0.435846  5462.03438   85.200147   46.551007  15.973224   
614  fd8ef6377f76  0.427300  2459.10720  130.138587   55.355778  10.005552   
615  fe1942975e40  0.363205  1263.53524   85.200147   23.685856   8.138688   
616  ffcca4ded3bb  0.482849  2672.53426  546.663930  112.006102   8.138688   

           AX        AY         AZ          BC  ...         FI 